In [5]:
import datetime
from selenium.webdriver.edge.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

def find_FTVN_news(n_hours_ago):

    options = Options()
    driver = webdriver.Edge(options=options)

    # 新聞網站 URL
    url = "https://www.ftvnews.com.tw/realtime/"
    driver.get(url)
    
    # 初始化一個空字典來存放時間
    news = {}

    index = 1

    # 用於存儲已找到的新聞標題
    finded_news_titles = set()

    # 持續滾動頁面,直到找不到符合時間範圍的新聞為止
    while True:

        # 找到所有新聞項目
        news_items = driver.find_elements(By.CLASS_NAME, 'content')

        # 如果沒有新的新聞項目,退出循環
        if not news_items:
            break
        
        # 標記本次循環是否找到了符合時間範圍的新聞
        found_new_news = False

        # 遍歷每個新聞項目
        for item in news_items:
            
            # 獲取標題
            title_element = item.find_element(By.CLASS_NAME, "title")
            news_title = title_element.text if title_element else ""
            
            # 如果標題已經被處理過,跳過
            if news_title in finded_news_titles:
                continue
            
            # 獲取時間
            try:
                time_element = item.find_element(By.CLASS_NAME, "time")
                news_time = time_element.get_attribute('data-time') if time_element else "" 
            except:
                continue
            
            # 判斷時間字串的格式是否與當前時間格式一致
            news_time = datetime.datetime.strptime(news_time, "%Y/%m/%d %H:%M:%S")
                
            # 如果新聞時間在n小時範圍內
            if n_hours_ago < news_time:
                found_new_news = True
                finded_news_titles.add(news_title)  # 將標題加入已處理集合

                link_element = item.find_element(By.CSS_SELECTOR, "div.content > a")
                news_link = link_element.get_attribute('href') if link_element else ""
                
                # print(f"標題: {news_title}")
                # print(f"連結: {news_link}")
                # print(f"時間: {news_time}")
                # print("-------------------")

                news[index] = [news_title, news_link, news_time]
                index += 1

            else:
                # 如果本次循環中沒有找到符合時間範圍的新聞,就退出
                break
        if not found_new_news:
            break
        
        # 模擬滾動到頁面底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # 等待頁面響應

    # 訪問每個新聞詳情頁,爬取內文
    for index in news:

        driver.get(news[index][1])

        # category_element = item.find_element(By.ID, "preface")
        # news_category = category_element.text if category_element else ""
        # news_category = news_category[:4]

        # category_element = driver.find_element_by_xpath('//script[contains(text(), "https://www.ftvnews.com.tw/tag/")]/following::script[@type="application/ld+json"]')
        # json_text = category_element.get_attribute("textContent")

        content_elements = driver.find_elements(By.ID, "contentarea")
        content_elements = driver.find_elements(By.CSS_SELECTOR, "p")
        news_content = "\n".join([p.text for p in content_elements])

        # news[index].append(news_category)
        news[index].append(news_content)
        # print(f"標題: {news[index][0]}")
        # # print(f"分類: {news[index][3]}")
        # print(f"內文: {news[index][-1]}") 

    return news
    
    #關閉瀏覽器
    driver.quit()

In [6]:
# find_FTVN_news(3)

Current date and time: 2024-03-16 23:40:23.600342
Current time: 23:40:23.600342
3 hours ago:2024-03-16 20:40:23.600342
標題: 凱特家庭照「修圖」風波　梅根開設新社群媒體帳號惹聯想
連結: https://www.ftvnews.com.tw/news/detail/2024316I17M1
時間: 2024-03-16 23:34:26
-------------------
標題: 獨家／中評網爆中國總理示警「國家將亡」　情治單位研判駭客全面啟動
連結: https://www.ftvnews.com.tw/news/detail/2024316W0209
時間: 2024-03-16 23:20:23
-------------------
標題: 李多慧從小愛跳舞！今圓夢出單曲「全程講中文」網驚：沒字幕也聽得懂
連結: https://www.ftvnews.com.tw/news/detail/2024316W0208
時間: 2024-03-16 23:06:15
-------------------
標題: 最正MC白色上衣太薄透！1秒曝光「暗黑內搭Bra」火辣畫面狂吸上萬讚
連結: https://www.ftvnews.com.tw/news/detail/2024316W0207
時間: 2024-03-16 23:02:01
-------------------
標題: 衛福部證實又接獲北市通報2件重大兒虐案　1死1重傷
連結: https://www.ftvnews.com.tw/news/detail/2024316W0205
時間: 2024-03-16 22:59:21
-------------------
標題: 快新聞／金正恩接班人確定了？　北朝鮮官媒為金主愛冠上「這稱號」
連結: https://www.ftvnews.com.tw/news/detail/2024316W0200
時間: 2024-03-16 22:52:34
-------------------
標題: 勞動部表揚績優勞工志工　肯定服務熱忱與貢獻
連結: https://www.ftvnews.com.tw/news/detail/20

{1: ['凱特家庭照「修圖」風波\u3000梅根開設新社群媒體帳號惹聯想',
  'https://www.ftvnews.com.tw/news/detail/2024316I17M1',
  datetime.datetime(2024, 3, 16, 23, 34, 26),
  '\n民視新聞／林昀嫻 綜合報導\n英國王室日前公布凱特王妃和孩子的家庭照，不過卻遭到美聯社等國際媒體撤下照片，指出照片經過「竄改」，對此凱特王妃也發聲明道歉。不過事件過沒多久，凱特的妯娌，也就是哈利王子的妻子梅根，卻在此時開設新的社群媒體帳號，時機巧合也引發外界更多揣測。\n英國節目記者：「網路上都在問一個問題『凱特在哪裡？』，每個人還在討論這張照片。」\n凱特王妃的「修照門」事件，過了快要一個星期，英國甚至海外媒體，都還緊抓著不放。\n英國節目主持人：「他們（王室）真的真的搞砸了，他們真的搞砸了他們搞砸了沒錯。」\n法新社照片主任巴拉達特：「我們現在的立場，會針對每張從肯辛頓宮釋出的照片，都會清楚且全面審視，（審視覺得沒問題）才會釋出給全球用戶。」\n修照門看似愈演愈烈，大西洋的另一頭，卻有另一場公關正在上演。睽違超過4年，凱特的妯娌，也就是哈利王子的妻子梅根，重新回歸社群媒體，14號開設名為美國里維埃拉果園（AmericanRivieraOrchard）的品牌帳號。\n\n\n'],
 2: ['獨家／中評網爆中國總理示警「國家將亡」\u3000情治單位研判駭客全面啟動',
  'https://www.ftvnews.com.tw/news/detail/2024316W0209',
  datetime.datetime(2024, 3, 16, 23, 20, 23),
  '\n民視新聞／蘇恩民報導\n立場親中的香港建置派媒體《中國評論通訊社》（簡稱中評社）今夜官網首頁刊出一則重大新聞，標題為「第十四屆全國人大會二會國務院總理李強政府工作報告」，引發國安單位高度關注，但因文章中李強嚴厲批評中國政權貪腐，甚至直言「國家將亡」，且記者署名「習禁評」，加上文章po出不到半小時就消失，情治系統研判中評社官網恐遭駭客入侵，不排除後續還有進一步攻擊行動。\n根據維基百科，中評社前身是1998年1月由時任海協會會長汪道涵定名、在香港創立的《中國評論》月刊，2000年獲准在中國公開銷